# Calibrate model to market implied volatility

In [22]:
import numpy as np
import plotly.graph_objects as go




In [23]:
strikes = np.linspace(80, 120, 30)
maturities = np.linspace(0.1, 2, 30)
S, T = np.meshgrid(strikes, maturities)
vol = 0.2 + 0.1*np.exp(-(S-100)**2/200) + 0.05*T #synthetic volatility surface



In [24]:
fig = go.Figure(data=[go.Surface(x=S, y=T, z=vol)])
fig.update_layout(title="Volatility Surface",
                  scene=dict(xaxis_title="Strike",
                             yaxis_title="Maturity",
                             zaxis_title="Vol"))
fig.show()

## A. Local Volatility Model

Vol is a function of price & time:

$$\sigma = \sigma(S,t)$$

MC simulation becomes:

$$dS = \mu S\,dt + \sigma(S,t)S\,dW$$

You need **local vol surface** (Dupire Formula), calibrated from the market.

We look at how option prices change with:

- **maturity** → $\partial C / \partial T$
- **strike** → $\partial^2 C / \partial K^2$

Those sensitivities contain information about volatility.

Dupire says:

$$\sigma^2_{\text{local}}(K,T) = \frac{\frac{\partial C}{\partial T}}{\frac{1}{2}K^2\frac{\partial^2 C}{\partial K^2}}$$




In [ ]:
# dupire :
# collect market sigma_iv(K,T)
# convert to prices C(K,T) plugging in BS 
# smooth surface
# compute sensitivities for option price wrt Ks and Ts
# apply dupire
# Convert K ↔ S for use in simulation 
    # At time t, the simulated spot is  S_t
    # We look up σ_local at K = S_t and maturity = t
    # (we just swap the labels)


In [38]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
expiry = ticker.options[0]             # one expiry for now
chain = ticker.option_chain(expiry)

calls = chain.calls                 
puts  = chain.puts
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL260102C00120000,2025-12-29 14:52:51+00:00,120.0,153.88,0.0,0.0,0.0,0.0,1.0,1,0.000010,True,REGULAR,USD
1,AAPL260102C00135000,2025-12-08 14:31:30+00:00,135.0,144.95,0.0,0.0,0.0,0.0,NaN,2,0.000010,True,REGULAR,USD
2,AAPL260102C00145000,2025-11-19 16:06:45+00:00,145.0,126.05,0.0,0.0,0.0,0.0,NaN,6,0.000010,True,REGULAR,USD
3,AAPL260102C00150000,2025-12-29 20:59:30+00:00,150.0,123.83,0.0,0.0,0.0,0.0,2.0,12,0.000010,True,REGULAR,USD
4,AAPL260102C00155000,2025-12-29 20:59:30+00:00,155.0,118.90,0.0,0.0,0.0,0.0,4.0,3,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,AAPL260102C00345000,2025-12-18 16:32:20+00:00,345.0,0.01,0.0,0.0,0.0,0.0,1.0,67,0.500005,False,REGULAR,USD
57,AAPL260102C00350000,2025-12-26 15:08:24+00:00,350.0,0.01,0.0,0.0,0.0,0.0,5.0,32,0.500005,False,REGULAR,USD
58,AAPL260102C00355000,2025-12-10 16:47:11+00:00,355.0,0.01,0.0,0.0,0.0,0.0,2.0,3,0.500005,False,REGULAR,USD
59,AAPL260102C00360000,2025-12-29 15:37:05+00:00,360.0,0.04,0.0,0.0,0.0,0.0,50.0,53,0.500005,False,REGULAR,USD


#### some comments:
- Many strikes/maturities have no liquidity 
- Some IVs are near 0 or extremely high



In [ ]:
spot = ticker.info['currentPrice'] 
spot  

273.76

In [29]:
len(ticker.options)

20

In [42]:
# plot a market surface:
maturities = np.array(ticker.options)
market_surface = {}
for expiry in maturities:
    chain = ticker.option_chain(expiry)
    strikes = chain.calls["strike"]
    IV = chain.calls["impliedVolatility"]
    market_surface[expiry] = dict(zip(strikes, IV))
    
    

In [43]:


strikes = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
maturities = list(market_surface.keys())

Z = np.zeros((len(maturities), len(strikes)))
for i,m in enumerate(maturities):
    for j,s in enumerate(strikes):
        Z[i,j] = market_surface[m].get(s, np.nan)

X, Y = np.meshgrid(strikes, maturities)

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(title="Volatility Surface",
                  scene=dict(xaxis_title="Strike",
                             yaxis_title="Maturity",
                             zaxis_title="Vol"))
fig.show()


In [ ]:
#Trim to minimum common strikes across maturities
min_len = min(len(v) for v in market_surface.values())

strikes_common = sorted(list(set(k for v in market_surface.values() for k in v.keys())))
strikes_common = strikes_common[:min_len]   # first min_len 

maturities = list(market_surface.keys())

Z = np.zeros((len(maturities), min_len))
for i,m in enumerate(maturities):
    sorted_strikes = sorted(market_surface[m].keys())[:min_len]
    for j,s in enumerate(sorted_strikes):
        Z[i,j] = market_surface[m][s]

X, Y = np.meshgrid(strikes_common, maturities)

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(scene=dict(
    xaxis_title="Strike",
    yaxis_title="Maturity",
    zaxis_title="Vol"))
fig.show()


#### Comments:
- mixing all strikes without aligning them properly ( same lenght but not same set of Ks)
- we could filter bid/ask zero contracts
- we could also restrict strikes near ATM (far strikes are messy)


## B. Heston stochastic volatility model

Vol follows its own random process:

$$dS = \mu S\,dt + \sqrt{v_t}S\,dW_1$$
$$dv_t = \kappa(\theta - v_t)\,dt + \sigma_v\sqrt{v_t}\,dW_2$$

- Two correlated random numbers per step
- Parameters calibrated to implied vol surface/market surface


## C. SABR model

Used a lot in interest rates:

$$dF = \sigma_t F^\beta\,dW_1$$
$$d\sigma = \alpha\sigma\,dW_2$$

Again, volatility is random → simulate both


Stress test prices under market shocks